<a href="https://colab.research.google.com/github/Timndg2030/Assignment-1/blob/master/Capstone_Project_Finding_the_Ideal_Restaurant_In_Cenral_London%2C_England.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project - Finding the Ideal Restaurant In Cenral London, England


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>


The aim of the project is to find the optimal location for a possible investment in opening a restaurant in Central London, England. The project will aim to find the optimal location. The aim will be to provide an insight into the most ideal location for a restaurant and the type of restaurant, in order to advice the Investment company to further approach the absolute franchise, which would be perfect for the location. Though Machine Learning we will be able to aggregate, wrangle data, along with the use of Foursquare to arrive at the sort out locations.

In the project, the **Data Section** will be a  description of the data that will be used to solve the problem and the sources. **Methodology** section will discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes. The
**Results** section will discuss the results and the 
Discussion Section will elaborate and discuss any observations noted and any recommendations suggested based on the results. Finally, the **Conclusion** section will report conclusion.









## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to the current restaurants in the neighborhood, if any
* distance of neighborhood from Oxford Circus

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Foursquare**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Central London will be obtained using **Foursquare** of well known Central London location (Oxford Circus)

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas in Cental London that have low restaurant density, particularly those with low number of restaurants. We will limit our analysis to area ~15 miles around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Central London** (Oxford Circus). 

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without a restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Anaysis


#### import Libraries

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from folium import CircleMarker
import os
from folium.plugins import FloatImage
print('Folium installed')
print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


In [ ]:
from folium import CircleMarker

### coordinates

In [ ]:
CLIENT_ID = '32HAJRP313FHYC5QLOFPTQL33QZADAH5PAUEPCLMF4QLFCSY'# your Foursquare ID
CLIENT_SECRET = 'EAYJPURSR1XBANJ2QUZSXZYSI0404MKCOUQ2MWPQY0IQDYV0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 32HAJRP313FHYC5QLOFPTQL33QZADAH5PAUEPCLMF4QLFCSY
CLIENT_SECRET:EAYJPURSR1XBANJ2QUZSXZYSI0404MKCOUQ2MWPQY0IQDYV0


In [ ]:
address = '352 Oxford St, London W1C 1JH'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.514665949999994 -0.14832092952281015


In [ ]:
search_query = 'restaurant'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

### General Location - Central London


In [ ]:
m = folium.Map(
    location=[51.514665949999994, -0.14832092952281015],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[51.514665949999994, -0.14832092952281015],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[51.514665949999994, -0.14832092952281015],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[51.514665949999994, -0.14832092952281015],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)


m

### Restaurants Filtered by Name and Coordiantes

In [ ]:
dataframe_filtered.name

0                                 Restaurant on 5
1                      Ascot's Restaurant and Bar
2                                 Kiri restaurant
3                      Mayfair pasta & restaurant
4                              Bonhams Restaurant
5                spanish tapas bar and restaurant
6               North Maroccan Bar And Restaurant
7                         Defune Sushi Restaurant
8                              Amjadia Restaurant
9         Holiday Inn London - Olivers restaurant
10                             deVille Restaurant
11                      Number 1 Bar & Restaurant
12                                Mews Restaurant
13                           Sotheby’s Restaurant
14                               Diwan Restaurant
15                           Belliaria Restaurant
16                              Kerala restaurant
17                         The Wallace Restaurant
18                         top restaurant mayfair
19          The Corner Restaurant & Champagne Bar


In [ ]:
# Requesting information of the Neighborhood
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
venues1 = results['response']['venues']
venues1 = pd.json_normalize(venues1)
venues1

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet,venuePage.id
0,526bd98211d29b555043e2c9,Restaurant on 5,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1594646899,False,334-348 Oxford St,51.515030,-0.148048,"[{'label': 'display', 'lat': 51.51503040765356...",44,GB,London,Greater London,United Kingdom,"[334-348 Oxford St, London, Greater London, Un...",NaN,NaN,NaN
1,4ac518c0f964a52080a320e3,Ascot's Restaurant and Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1594646899,False,350 Oxford St.,51.514865,-0.148562,"[{'label': 'display', 'lat': 51.51486532013419...",27,GB,London,Greater London,United Kingdom,"[350 Oxford St., London, Greater London, W1C 1...",W1C 1BY,NaN,NaN
2,581fa5951ff7964776c2dfa2,Kiri restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1594646899,False,22 Woodstock Road,51.514000,-0.147184,"[{'label': 'display', 'lat': 51.514, 'lng': -0...",108,GB,London,Greater London,United Kingdom,"[22 Woodstock Road, London, Greater London, WC...",WC1 2AR,NaN,NaN
3,4cf666ce1801a1432561e9d4,Mayfair pasta & restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1594646899,False,61 Davies Street,51.513253,-0.148659,"[{'label': 'display', 'lat': 51.51325343882792...",158,GB,London,Greater London,United Kingdom,"[61 Davies Street, London, Greater London, W1K...",W1K 5JL,NaN,NaN
4,5673fe7e498e84771c1a7836,Bonhams Restaurant,"[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",v-1594646899,False,7 Haunch Of Venison Yard,51.513461,-0.146436,"[{'label': 'display', 'lat': 51.513461, 'lng':...",187,GB,NaN,NaN,United Kingdom,"[7 Haunch Of Venison Yard, United Kingdom]",NaN,NaN,NaN
5,4be40666d4f7c9b6278c2320,Defune Sushi Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1594646899,False,34 George St,51.517771,-0.153638,"[{'label': 'display', 'lat': 51.51777118506121...",505,GB,Paddington,Greater London,United Kingdom,"[34 George St, Paddington, Greater London, W1U...",W1U 7DP,NaN,NaN
6,53d5684b498e97994746e25e,spanish tapas bar and restaurant,"[{'id': '4bf58dd8d48988d14d941735', 'name': 'P...",v-1594646899,False,NaN,51.515243,-0.150697,"[{'label': 'display', 'lat': 51.515243, 'lng':...",176,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,NaN
7,4ac518e5f964a520fbaa20e3,Amjadia Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594646899,False,15 Picton Pl.,51.515363,-0.151075,"[{'label': 'display', 'lat': 51.51536332547299...",206,GB,London,Greater London,United Kingdom,"[15 Picton Pl., London, Greater London, W1U 1B...",W1U 1BW,NaN,NaN
8,504cda7fe4b05ebf6e0bf945,North Maroccan Bar And Restaurant,"[{'id': '4bf58dd8d48988d1c3941735', 'name': 'M...",v-1594646899,False,NaN,51.515636,-0.151076,"[{'label': 'display', 'lat': 51.515636, 'lng':...",219,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,NaN
9,56890327498e2cded86da6c2,Holiday Inn London - Olivers restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1594646899,False,57 - 59 Welbeck,51.516963,-0.149758,"[{'label': 'display', 'lat': 51.51696313289656...",274,GB,London,Greater London,United Kingdom,"[57 - 59 Welbeck, London, Greater London, Uni...",NaN,NaN,NaN


### Visual representation of Restaurants 

In [89]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15.5) # generate map centred around the Oxford Circus

# add a red circle marker to represent the Oxford Circus
folium.CircleMarker(
    [latitude, longitude],
    radius=11,
    color='red',
    popup='Oxford Circus',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.7
).add_to(venues_map)

# add the  restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map


## Results and Discussion


From the analysis above, there is a clear imbalance in the distribution of restaurants in Central London, with most restaurants on one side. There are around ~39 in our initial area of interest around Oxford Circus. The highest concentration of restaurants was detected north and west from of Oxford Circus.


Those locations indicated are clustered to create a zone of interest which contain the greatest number of location candidates. Addresses of  the restaurants were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.
Results contains a largest number of potential new restaurant locations based on the number of and distance to existing venues - both restaurants in general and location. 

It does not imply that those are without restaurants should are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Oxford Circus, but not crowded with existing restaurants - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended locations should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.




## Conclusion

The purpose of this project was to identify the optimal area close to Central London Oxford Circus in order to aid an investor in narrowing down the search. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Final decision on optimal restaurant location will be made by the investor based on specific characteristics of the restaurant and and locations , taking into consideration additional factors like attractiveness of each location and the reasons for the lack of restaurants in those areas where there were none identified .